In [ ]:
import numpy as np
import tensorflow as tf
import os

import librosa
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

opj = os.path.join

In [2]:
gtzan_path = r'datasets\gtzan'
genres_dir = opj(gtzan_path, 'genres_original')

In [3]:
def load_data_by_classes(dataset_path):
    data = []
    labels = []
    classes = os.listdir(dataset_path)
    for cl in tqdm(classes):
        full_p = opj(dataset_path, cl)
        for file in os.listdir(full_p):
            try:
                wave, _ = librosa.load(opj(full_p, file))
            except:
                continue
            data.append(wave)
            labels.append(cl)
    
    return data, labels

In [119]:
data, labels = load_data_by_classes(genres_dir)

100%|████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.54it/s]


In [120]:
sizes = [len(item) for item in data]

In [121]:
min_size = min(sizes)

In [122]:
data = [item for item in data]

# int(min_size/16)

In [123]:
data = np.array(data)
data = data[..., np.newaxis]


le = LabelEncoder()
labels = le.fit_transform(labels)


C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [124]:
from sklearn.model_selection import train_test_split

In [125]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.15)

In [126]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [127]:
X_train.shape

(679, 1)

In [129]:
data.shape

(999, 1)

In [130]:
data = tf.convert_to_tensor(data)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [128]:
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [106]:
# data = tf.convert_to_tensor(data)
# labels_one_hot = tf.one_hot(labels, depth=10)

In [ ]:
from tensorflow.keras.layers import Conv1D, Dropout, LSTM, TimeDistributed, Activation, Dense, Input, MaxPooling1D, Lambda, Flatten

In [ ]:
N_LAYERS = 4
CONV_FILTER_COUNT = 64
FILTER_LENGTH = 25

GENRES = 10

In [43]:
data.shape

TensorShape([999, 41250, 1])

In [44]:
import tensorflow.keras.backend as K

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import RMSprop

In [45]:
LSTM_COUNT = 256
BATCH_SIZE = 32
EPOCH_COUNT = 80

kernel_size = 100

In [49]:
model = Sequential()

model.add(Conv1D(32, kernel_size, input_shape=data.shape[1:], activation='relu'))
model.add(Conv1D(32, kernel_size, activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=4)) 

model.add(Conv1D(32, kernel_size, activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=4)) 

model.add(Flatten())
model.add(Dense(100, activation='relu')) 
# model.add(BatchNormalization())
model.add(Dense(10 ,activation='softmax'))

model.compile(loss='categorical_crossentropy' , optimizer=RMSprop(learning_rate=0.0001) , metrics=['accuracy'])

In [50]:
model.fit(data, labels_one_hot, epochs = 24)

Epoch 1/24
32/32 [==============================] - 16s 500ms/step - loss: 2.2740 - accuracy: 0.1231
Epoch 2/24
32/32 [==============================] - 16s 500ms/step - loss: 2.1270 - accuracy: 0.2422
Epoch 3/24
32/32 [==============================] - 16s 500ms/step - loss: 1.8398 - accuracy: 0.3724
Epoch 4/24
32/32 [==============================] - 16s 501ms/step - loss: 1.5915 - accuracy: 0.4625
Epoch 5/24
32/32 [==============================] - 16s 502ms/step - loss: 1.3411 - accuracy: 0.5756
Epoch 6/24
32/32 [==============================] - 16s 502ms/step - loss: 1.1075 - accuracy: 0.6557
Epoch 7/24
32/32 [==============================] - 16s 502ms/step - loss: 0.8436 - accuracy: 0.7548
Epoch 8/24
32/32 [==============================] - 16s 502ms/step - loss: 0.5846 - accuracy: 0.8448
Epoch 9/24
32/32 [==============================] - 16s 503ms/step - loss: 0.4410 - accuracy: 0.9009
Epoch 10/24
32/32 [==============================] - 16s 508ms/step - loss: 0.3392 - accura

In [23]:
inp = Input((None, data.shape[1]))

In [24]:
conv1 = Conv1D(filters=64, kernel_size=(FILTER_LENGTH,))(inp)
mp1 = MaxPooling1D(2)(conv1)
act1 = Activation('relu')(mp1)


conv2 = Conv1D(filters=128, kernel_size=(FILTER_LENGTH,))(act1)
mp2 = MaxPooling1D(2)(conv2)
act2 = Activation('relu')(mp1)


conv3 = Conv1D(filters=256, kernel_size=(FILTER_LENGTH,))(act2)
mp3 = MaxPooling1D(2)(conv3)
act3 = Activation('relu')(mp3)

layer = Dropout(0.5)(act3)
layer = LSTM(LSTM_COUNT, return_sequences=True)(layer)
layer = Dropout(0.5)(layer)
layer = TimeDistributed(Dense(GENRES))(layer)
layer = Activation('softmax', name='output_realtime')(layer)


In [25]:
time_distributed_merge_layer = Lambda(
        function=lambda x: K.mean(x, axis=1),
        output_shape=lambda shape: (shape[0],) + shape[2:],
        name='output_merged')

model_output = time_distributed_merge_layer(layer)
model = Model(inp, model_output)
opt = RMSprop(lr=0.00001)  # Optimizer
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])

In [ ]:
model.fit(data, labels_one_hot)

In [ ]:
inp = Input((data.shape[1]))

layer = inp

for i in range(N_LAYERS):
    # Convolutional layer names are used by extract_filters.py
    layer = Conv1D(
        filters=(None, CONV_FILTER_COUNT),
        kernel_size=(FILTER_LENGTH,),
        name='convolution_' + str(i + 1))(layer)
    layer = Activation('relu')(layer)
    layer = MaxPooling1D(2)(layer)

layer = Dropout(0.5)(layer)
layer = LSTM(LSTM_COUNT, return_sequences=True)(layer)
layer = Dropout(0.5)(layer)
layer = TimeDistributed(Dense(len(GENRES)))(layer)
layer = Activation('softmax', name='output_realtime')(layer)
time_distributed_merge_layer = Lambda(
    function=lambda x: K.mean(x, axis=1),
    output_shape=lambda shape: (shape[0],) + shape[2:],
    name='output_merged')
model_output = time_distributed_merge_layer(layer)
model = Model(model_input, model_output)


In [ ]:
opt = RMSprop(lr=0.00001)  # Optimizer
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])